# DIverse Counterfactual Explanations (DICE)

In [1]:
!pip install dice-ml --quiet

In [2]:
import dice_ml
from dice_ml.utils import helpers
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

ATTENTION ! Ici on charge les données avec google drive, il faut modifier si ce n'es pas le cas

In [3]:
url = "https://raw.githubusercontent.com/AngeloLF/M2_MALIA_XAI/refs/heads/main/data/hws.csv" # si pas dispo assez sur https://phl.upr.edu/hwc/data, ou sur le github https://github.com/AngeloLF/M2_MALIA_XAI
df = pd.read_csv(url)
df = df.dropna(subset=["P_TEMP_SURF"])

# categorical_cols = ["P_TYPE_TEMP"] # P_TYPE

# for cate in categorical_cols:
#     one_hot = pd.get_dummies(df[cate]).astype("int64")
#     df = df.join(one_hot)
#     df = df.drop(columns=cate)

x = df[['P_MASS', "P_RADIUS", "P_FLUX", "S_DISTANCE", "P_TEMP_SURF", "S_TEMPERATURE"]] # , "Subterran", "Superterran", "Terran", "Cold", "Hot", "Warm"
x = x.select_dtypes(exclude=["object"])
y = df["P_HABITABLE"]
x.head()

,P_MASS,P_RADIUS,P_FLUX,S_DISTANCE,P_TEMP_SURF,S_TEMPERATURE
2,5.10,2.110,238.528680,1073.7600,1130.60440,5698.0
4,4.94,2.070,51.163853,367.0000,769.42507,5240.0
13,3.40,1.402,766.125920,80.5407,1504.26870,5059.0
22,7.32,2.180,48.505343,51.9503,746.01801,3709.0
27,7.25,2.160,492.983910,45.6830,1354.17430,5863.0


In [4]:
print(np.unique(y.values, return_counts=True))
y[y == 2] = 1
print(np.unique(y.values, return_counts=True))

(array([0, 1, 2]), array([2371,   29,   41]))
(array([0, 1]), array([2371,   70]))


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
# Entraîner un modèle RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [7]:
# Vérifier l'accuracy
accuracy = model.score(x_test, y_test)
print(len(y_test))
print(f"Accuracy du modèle : {accuracy:.2f}")

489
Accuracy du modèle : 1.00


In [8]:
continuous_features = x_train.columns.tolist()

# Créer l'objet Data pour DiCE
d = dice_ml.Data(dataframe=pd.concat([x_train, y_train], axis=1),
                 continuous_features=continuous_features,
                 outcome_name='P_HABITABLE')  # Nom de la colonne cible

m = dice_ml.Model(model=model, backend="sklearn", model_type="classifier")

In [9]:
exp = dice_ml.Dice(d, m, method="random")

# Sélectionner une exoplanète à expliquer
query_instance = x_test.dropna().iloc[4:5]

# Générer 2 explications contrefactuelles
explanations = exp.generate_counterfactuals(query_instance, total_CFs=2, desired_class="opposite", random_seed=6666)

# Afficher les explications
explanations.visualize_as_dataframe()

100%|██████████| 1/1 [00:01<00:00,  1.11s/it]

Query instance (original outcome : 1)


,P_MASS,P_RADIUS,P_FLUX,S_DISTANCE,P_TEMP_SURF,S_TEMPERATURE,P_HABITABLE
0,5.2,2.13,0.270091,7.61778,198.928467,3728.0,1



Diverse Counterfactual set (new outcome: 0)


,P_MASS,P_RADIUS,P_FLUX,S_DISTANCE,P_TEMP_SURF,S_TEMPERATURE,P_HABITABLE
0,355.11,2.13,0.270091,7.61778,2308.92518,3728.0,0
1,5.20,2.13,0.270091,7.61778,5214.89301,3728.0,0


In [10]:
exp = dice_ml.Dice(d, m, method="random")

# Sélectionner une exoplanète à expliquer
query_instance = x_test.dropna().iloc[14:15]


# Générer 2 explications contrefactuelles
features_to_vary = ["P_FLUX", "S_DISTANCE", "P_TEMP_SURF", "S_TEMPERATURE"]
explanations = exp.generate_counterfactuals(query_instance, total_CFs=2, desired_class="opposite", random_seed=666, features_to_vary=features_to_vary)

# Afficher les explications
explanations.visualize_as_dataframe()

100%|██████████| 1/1 [00:00<00:00,  1.12it/s]

Query instance (original outcome : 0)


,P_MASS,P_RADIUS,P_FLUX,S_DISTANCE,P_TEMP_SURF,S_TEMPERATURE,P_HABITABLE
0,1.24,1.07,917.122009,623.901001,1583.187256,6112.0,0



Diverse Counterfactual set (new outcome: 1)


,P_MASS,P_RADIUS,P_FLUX,S_DISTANCE,P_TEMP_SURF,S_TEMPERATURE,P_HABITABLE
0,1.24,1.07,917.122,623.901000,316.43202,6112.0,1
1,1.24,1.07,917.122,65.985847,280.15917,6112.0,1
